In [4]:
!pip install tensorflow_decision_forests

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_decision_forests-1.8.1.tar.gz (15.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [2]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
df= pd.read_csv('train.csv')

NameError: name 'pd' is not defined

In [ ]:
df

In [ ]:
plot_df = df.Transported.value_counts()
plot_df.plot(kind="bar")

fig, ax = plt.subplots(5,1,  figsize=(10, 10))
plt.subplots_adjust(top = 2)

sns.histplot(df['Age'], color='b', bins=50, ax=ax[0]);
sns.histplot(df['FoodCourt'], color='b', bins=50, ax=ax[1]);
sns.histplot(df['ShoppingMall'], color='b', bins=50, ax=ax[2]);
sns.histplot(df['Spa'], color='b', bins=50, ax=ax[3]);
sns.histplot(df['VRDeck'], color='b', bins=50, ax=ax[4]);

In [ ]:
headers=[['PassengerId','HomePlanet','CryoSleep','Cabin','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Name']]

In [ ]:
missing_data=df.isnull()
df.head(15)

In [ ]:
missing_data["Cabin"].value_counts()

In [ ]:
missing_data["Age"].value_counts()

In [ ]:
df['Cabin'].value_counts()

In [ ]:
df.dropna(subset=['Cabin'], axis=0, inplace=True)

In [ ]:
df[["Deck", "Cabin_num", "Side"]] = df["Cabin"].str.split("/", expand=True)

In [ ]:
df = df.drop('Cabin', axis=1)

In [ ]:
df.info()

In [ ]:
#We are replacing the nan values present in the following columns with mean value of that column
avg_Roomservice = df["RoomService"].astype('float').mean(axis=0)
#axis = 0, means along the rows
avg_Roomservice

In [ ]:
df["RoomService"].replace(np.nan, avg_Roomservice, inplace=True)

In [ ]:
avg_Foodcourt = df["FoodCourt"].astype('float').mean(axis=0)

In [ ]:
df["FoodCourt"].replace(np.nan,avg_Foodcourt , inplace=True)

In [ ]:
avg_ShoppingMall = df["ShoppingMall"].astype('float').mean(axis=0)

In [ ]:
df["ShoppingMall"].replace(np.nan,avg_ShoppingMall , inplace=True)

In [ ]:
avg_Spa = df["Spa"].astype('float').mean(axis=0)

In [ ]:
df["Spa"].replace(np.nan,avg_Spa , inplace=True)

In [ ]:
avg_VRDeck = df["VRDeck"].astype('float').mean(axis=0)

In [ ]:
df["VRDeck"].replace(np.nan,avg_VRDeck , inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#most_freuent was used beacause age should be whole nos
imputer.fit(df[['HomePlanet','CryoSleep','Destination','Age','VIP']])
df[['HomePlanet','CryoSleep','Destination','Age','VIP']] = imputer.transform(df[['HomePlanet','CryoSleep','Destination','Age','VIP']])

In [ ]:
df.drop('Name',axis=1, inplace=True)
df.drop('PassengerId',axis=1, inplace=True)

In [ ]:
missing_data=df.isnull()

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

In [ ]:
df['Transported'] = df['Transported'].astype(int)
df['VIP'] = df['VIP'].astype(int)
df['CryoSleep'] = df['CryoSleep'].astype(int)

In [ ]:
df.head(10)

In [ ]:
df['Destination'].unique()
df['HomePlanet'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

In [ ]:
#Using label encoder for converting into numeric data
data = {'Destination':['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e']}
df['Destination']=le.fit_transform(df['Destination'])

data = {'HomePlanet':['Europa', 'Earth', 'Mars']}
df['HomePlanet']=le.fit_transform(df['HomePlanet'])

df.head(10)

In [ ]:
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

In [ ]:
label='Transported'

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label)

In [ ]:
tfdf.keras.RandomForestModel()

In [ ]:
rf = tfdf.keras.RandomForestModel()

In [ ]:
rf.fit(x=train_ds)

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)

In [ ]:
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")
plt.show()

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

In [ ]:
evaluation = rf.evaluate(x=valid_ds,return_dict=True)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

In [ ]:
test_df = pd.read_csv('test.csv')
submission_id = test_df.PassengerId
test_df[['VIP', 'CryoSleep']] = test_df[['VIP', 'CryoSleep']].fillna(value=0)
test_df[["Deck", "Cabin_num", "Side"]] = test_df["Cabin"].str.split("/", expand=True)
test_df = test_df.drop('Cabin', axis=1)
test_df['VIP'] = test_df['VIP'].astype(int)
test_df['CryoSleep'] = test_df['CryoSleep'].astype(int)
data = {'Destination':['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e']}
test_df['Destination']=le.fit_transform(test_df['Destination'])
data = {'HomePlanet':['Europa', 'Earth', 'Mars']}
test_df['HomePlanet']=le.fit_transform(test_df['HomePlanet'])
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)
predictions = rf.predict(test_ds)
n_predictions = (predictions > 0.5).astype(bool)
output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions.squeeze()})

output.head()